# Quickstart/Cheat-Sheet

<div class="alert alert-warning">

Warning

`pyglotaran` is in very early stage of development. You should not use it for actual science at the moment.

</div>

To start using `pyglotaran` in your project, you have to import it first.
In addition we need to import some extra components for later use.

In [ ]:
import glotaran as gta
from glotaran.analysis.optimize import optimize
from glotaran.analysis.scheme import Scheme

Let us get some example data to analyze:

In [ ]:
from glotaran.examples.sequential import dataset

dataset

Like all data in `pyglotaran`, the dataset is a [xarray.Dataset](https://xarray.pydata.org/en/stable/api.html#dataset).
You can find more information about the ``xarray`` library the [xarray hompage](http://xarray.pydata.org/en/stable/).

The loaded dataset is a simulated sequential model.

Now we lets plot some time traces.

In [ ]:
plot_data = dataset.data.sel(spectral=[620, 630, 650], method="nearest")
plot_data.plot.line(x="time", aspect=2, size=5);

We can also plot spectra at different times.

In [ ]:
plot_data = dataset.data.sel(time=[1, 10, 20], method="nearest")
plot_data.plot.line(x="spectral", aspect=2, size=5);

To get an idea about how to model your data, you should inspect the singular
value decomposition. Pyglotaran has a function to calculate it (among other
things).

In [ ]:
dataset = gta.io.prepare_time_trace_dataset(dataset)
dataset

First, take a look at the first 10 singular values:

In [ ]:
plot_data = dataset.data_singular_values.sel(singular_value_index=range(0, 10))
plot_data.plot(yscale="log", marker="o", linewidth=0, aspect=2, size=5);

To analyze our data, we need to create a model. 

Create a file called `model.yaml` in your working directory and fill it with the following:
```yaml
type: kinetic-spectrum

initial_concentration:
 input:
   compartments: [s1, s2, s3]
   parameters: [input.1, input.0, input.0]

k_matrix:
 k1:
   matrix:
     (s2, s1): kinetic.1
     (s3, s2): kinetic.2
     (s3, s3): kinetic.3

megacomplex:
 m1:
   k_matrix: [k1]

irf:
 irf1:
   type: gaussian
   center: irf.center
   width: irf.width

dataset:
 dataset1:
   initial_concentration: input
   megacomplex: [m1]
   irf: irf1
```

Now you can load the model file.

In [ ]:
model = gta.read_model_from_yaml_file("model.yaml")

You can check your model for problems with `model.validate`.

In [ ]:
model.validate()

Now define some starting parameters. Create a file called `parameters.yaml` with
the following content.

```yaml
input:
 - ['1', 1, {'vary': False, 'non-negative': False}]
 - ['0', 0, {'vary': False, 'non-negative': False}]

kinetic: [
    0.5,
    0.3,
    0.1,
]

irf:
 - ['center', 0.3]
 - ['width', 0.1]
```

In [ ]:
parameters = gta.read_parameters_from_yaml_file("parameters.yaml")

You can `model.validate` also to check for missing parameters.

In [ ]:
model.validate(parameters=parameters)

Since not all problems in the model can be detected automatically it is wise to
visually inspect the model. For this purpose, you can just print the model.

In [ ]:
print(model)

The same way you should inspect your parameters.

In [ ]:
print(parameters)

Now we have everything together to optimize our parameters. First we import optimize.

In [ ]:
scheme = Scheme(model, parameters, {"dataset1": dataset})
result = optimize(scheme)
print(result)
print(result.optimized_parameters)

You can get the resulting data for your dataset with `result.get_dataset`.

In [ ]:
result_dataset = result.get_dataset("dataset1")
result_dataset

The resulting data can be visualized the same way as the dataset. To judge the
quality of the fit, you should look at first left and right singular vectors of
the residual.

In [ ]:
residual_left = result_dataset.residual_left_singular_vectors.sel(left_singular_value_index=0)
residual_right = result_dataset.residual_right_singular_vectors.sel(right_singular_value_index=0)
residual_left.plot.line(x="time", aspect=2, size=5)
residual_right.plot.line(x="spectral", aspect=2, size=5);

Finally, you can save your result.

In [ ]:
result_dataset.to_netcdf("dataset1.nc")